### Generate download script for DN/KB


In [2]:
import pandas as pd
import requests
import json
import traceback
import time
import re
import colorama
import random
from colorama import Fore, Style
INPUT_FILE='dn.csv'
df = pd.read_csv(INPUT_FILE, index_col=0, parse_dates=['date_published'])

In [8]:
## COLORIZATION CODE FOR OUTPUT
bad_colors = ['BLACK', 'WHITE', 'LIGHTBLACK_EX', 'RESET']
#bad_colors = []
codes = vars(colorama.Fore)
colors = [codes[color] for color in codes if color not in bad_colors]
##/

print("Total issues (dark-packages):", len(df.index))
length = len(df.index)
LIMIT = 10
#length = LIMIT
print("Issues to parse:", length)
issue_list = []

for i in range(length):
    row = df.iloc[i]
    dark_id = row['id']
    title = row['title']
    date_published = row['date_published']
    
    #Set up and do GET request
    headers = {'accept': 'application/json'}
    url = f'https://data.kb.se/{dark_id}'
    #print(f'(idx: {i} / title: {title})', end='')
    print('.',end='')
    
    # !? had to disable verify certificate, something strange going on... !?
    requests.packages.urllib3.disable_warnings()
    response = requests.get(url, headers=headers, verify=False)
    
    
    def wait(t,i):
        '''This function is called periodically and sleeps to avoid too many http requests'''
        time.sleep(t)
        #print('+',end='')
        print(random.choice(colors)+'+', end='')

        False if i % 60 else print(f' :{i}\n',end='')
        return
    
    try:
        # if and modulo calls wait() every Nth request
        False if i % 20 else wait(1,i)
        json_response = json.loads(response.text)
    except:
        traceback.print_exc()
        print(json_response, response.text, response, response.headers)
        break
    l = json_response['includes']
    item_dict = {'title': '',
                 'fileurl': '',
                 'filename': '',
                 'date': '',
                 'contentSize': '',
                 'md5': '',
                 'dark_id': ''
                 }
    issue = []
    
    for i in range(len(l)):
        fileurl = l[i]['@id']
        filename = l[i]['fileName']
        contentsize = l[i]['contentSize']
        md5 = l[i]['checksum']['value']
        item_dict = {'title': title,
                     'fileurl': fileurl,
                     'filename': filename,
                     'date': date_published,
                     'contentSize': contentsize,
                     'md5': md5,
                     'dark_id': dark_id
                     }
        issue.append(item_dict.copy())
        pass
    issue_list.append(issue.copy())
    pass
pass
print("\nLength of issue_list:",len(issue_list))
# print(issue_list[1][0]['fileurl'])
# print(issue_list[1][1]['fileurl'])
# print(issue_list[1][2]['fileurl'])
# print(issue_list[1][3]['fileurl'])
# print(issue_list[1][4]['fileurl'])
# print(issue_list[1][5]['fileurl'])
# print(issue_list[1][6]['fileurl'])

Total issues (dark-packages): 10000
Issues to parse: 10000
.+ :0
....................+....................+....................+ :60
....................+....................+....................+ :120
....................+....................+....................+ :180
....................+....................+....................+ :240
....................+....................+....................+ :300
....................+....................+....................+ :360
....................+....................+....................+ :420
....................+....................+....................+ :480
....................+....................+....................+ :540
....................+....................+....................+ :600
....................+....................+....................+ :660
....................+....................+....................+ :720
....................+....................+....................+ :780
....................+..................

In [10]:
from hurry.filesize import size
total_size=0
textonly_size=0
activated_size=0
DO_COMMENT=''
OUTPUT_FILE=r'wget_dn.sh'
SET_ONLY_TEXT=True
START_YEAR=1888
END_YEAR=1891
deactivated=0
count=0
try:
    with open(OUTPUT_FILE, 'w') as fp:
        fp.write("#!/bin/bash\n")
        #fp.write("%s\n" % command)

        for ii in range(len(issue_list)):
            #print("Issue dl commands:")
            print(random.choice(colors)+'+', end='')
            False if ii % 500 else print(f'\n', end='')
            
            #print(colors[ii % len(colors)]+'+', end='')
            for i in range(len(issue_list[ii])):
                # print(issue_list[0][i]['fileurl'])
                current_dict = issue_list[ii][i]
                title = current_dict['title']
                fileurl = current_dict['fileurl']
                filename = current_dict['filename']
                date = current_dict['date']
                contentsize = current_dict['contentSize']
                md5 = current_dict['md5']
                dark_id = current_dict['dark_id']
                clean_title = re.sub(r'[0-9]+', '', title.capitalize())
                clean_title = re.sub(r'-+', '', clean_title)
                clean_title = re.sub(r'\s+', '', clean_title)
                sep = '_'
                
                year = str(date.year)
                month = str(date.month).zfill(2)
                day = str(date.day).zfill(2)
                new_filename = clean_title+sep \
                    + year+sep \
                    + month+sep \
                    + day+sep \
                    + filename
                command = 'wget --continue -O \"'+clean_title.lower()+'/'+year+'/'+new_filename+'\" '+fileurl
                prefix = 'mkdir -p '+clean_title.lower()+'/'+year+'/ && '
                suffix = f' # {size(contentsize)}'
                
                if re.search('jp2', filename) and SET_ONLY_TEXT:
                    DO_COMMENT='#';deactivated=deactivated+1
                elif date.year < START_YEAR or date.year >END_YEAR:
                    DO_COMMENT = '#'
                    deactivated = deactivated+1
                else: DO_COMMENT=''
                if not DO_COMMENT:
                    activated_size = activated_size+contentsize
                if re.search('xml', filename):
                    textonly_size=textonly_size+contentsize
                
                total_size = total_size+contentsize
                command = DO_COMMENT+prefix+command+suffix
                #All set – print the wget command for the download batch:
                #print(command)

                fp.write("%s\n" % command)
                count = count+1
except:
    print("-- Error!")
else:
    print(f'\n++ Wrote {count} lines as "{OUTPUT_FILE}" ({deactivated} deactivated and {count-deactivated} activated lines)')
            
print(f'Total size of dataset: {size(total_size)}')
print(f'Text size in dataset: {size(textonly_size)}')
print(f'Selected subset ({START_YEAR}-{END_YEAR} size): {size(activated_size)}')


+
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++